# Runner

> Runner object to run the request collection as well as response. We have the callback implementaion here.

In [ ]:
#| export

from exp.nb_ReqRespBunch import *

In [ ]:
#| export

def listify(o):
    if o is None: return []
    if isinstance(o, list): return o
    if isinstance(o, str): return [o]
    if isinstance(o, Iterable): return list(o)
    return [o]

In [ ]:
#| export
import re
class Callback():
    _order=0
    def set_runner(self, run): self.run=run
    def __getattr__(self, k): return getattr(self.run, k)
    
    @property
    def name(self):
        name = re.sub(r'Callback$', '', self.__class__.__name__)
        return name
    
    def __call__(self, cb_name):
        f = getattr(self, cb_name, None)
        if f and f(): return True
        return False

In [ ]:
#| export
from tqdm.notebook import tqdm

class Runner():
    def __init__(self, cb_funcs=None):
        base_cb = [CookieCallback, ResponseStoreCallback]
        if not cb_funcs:
            cb_funcs = base_cb
        else:
            cb_funcs = base_cb + cb_funcs
        cbs = []
        for cfs in listify(cb_funcs):

            cb = cfs()
            setattr(self, cb.name, cb)
            cbs.append(cb)
        self.cbs = cbs
    
    @property
    def reqs_bunch(self):  return self.bunchs.reqs

    @property
    def resps_bunch(self):  return self.bunchs.resps

    @property
    def kwargs_bunch(self):  return self.bunchs.kwargs

    def set_kwargs(self, idx, val):
        self.bunchs.set_kwargs(idx, val)

    def set_resp(self, idx, val):
        self.bunchs.set_resp(idx, val)
    
    def send_one_req(self, req:RestReq, **kwargs):
            #print("insideof send_one_req")
            #print(kwargs)
            self.resp = req()(**kwargs)
            #print(self.resp)
    def run(self, bunchs):
        self.bunchs, self.epoch  = bunchs,0

        for cb in self.cbs: cb.set_runner(self)

        self('before_run')
        
        for i  in  tqdm(range(len(self.reqs_bunch))):
            
            self.req = self.reqs_bunch[i]
            self.epoch = i
            
            self('before_request') 
            
            kwargs = self.kwargs_bunch[i] 
            
            self.reqs_bunch[i].set_req_kwargs(kwargs) 
            
            if kwargs:
                self.send_one_req(self.req, **kwargs)
            else:
                self.send_one_req(self.req)


            self('after_request') 
            
            
        
        self('after_run')

    def __call__(self, cb_name):
        res = False
        for cb in sorted(self.cbs, key=lambda x: x._order): res = cb(cb_name) or res
        return res

In [ ]:
#| export
class CookieCallback(Callback):
    def before_request(self):
        
        if self.epoch != 0:
            self.set_kwargs(self.epoch, ('cookies', self.resps_bunch[self.epoch - 1].cookies))
        
    def after_request(self):
        #self.set_kwargs(self.epoch, ('cookies', self.resp.cookies))
        #print(f"{self.epoch=} {self.kwargs_bunch=}")
        ...
class ResponseStoreCallback(Callback):
    def after_request(self):
        self.set_resp(self.epoch, self.resp)

In [ ]:
url = lambda: f"{get_env('url1')}/get"

params = lambda : {
    "foo1": f"{get_env('foo1')}",
    "foo2": f"{get_env('foo2')}",
}

# Create the RestReqFactory instance
req1 = RestReqFactory(
    method="GET",
    url_provider=url,
    params_provider=params,
)

In [ ]:
ENV['foo1'] = 'foo1'
ENV['foo2'] = 'foo2'
ENV['url1']  = 'https://postman-echo.com'
ENV['url2']  = 'https://reqres.in'

In [ ]:
url = lambda : f"{get_env('url2')}/api/users/2"

req2 = RestReqFactory(
    method="GET",
    url_provider=url,
)

In [ ]:
reqs = ReqRespBunch([req1, req2])

In [ ]:
runner = Runner()

In [ ]:
runner.run(reqs)

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print_req_resp(reqs)

[{'request': {
      "method": "GET",
      "url": "https://postman-echo.com/get",
      "headers": "None",
      "params": {
          "foo1": "foo1",
          "foo2": "foo2"
      },
      "body": "None",
      "kwargs": "None"
  },
  'response': {
      "url": "https://postman-echo.com/get?foo1=foo1&foo2=foo2",
      "status": 200,
      "headers": {
          "Date": "Sat, 08 Jun 2024 15:34:04 GMT",
          "Content-Type": "application/json; charset=utf-8",
          "Content-Length": "438",
          "Connection": "close",
          "ETag": "W/\"1b6-9BIXzYxXp0/QGNIuzYKsV6l1t58\"",
          "set-cookie": "sails.sid=s%3Asxwk3G05JAOj5Lq7gEoNIKpAT5qQRMsL.681C3Yx6JxWK7cdRgpg53qChIfhOTK9un5Cs0fRdOq4; Path=/; HttpOnly"
      },
      "cookies": {
          "sails.sid": "s%3Asxwk3G05JAOj5Lq7gEoNIKpAT5qQRMsL.681C3Yx6JxWK7cdRgpg53qChIfhOTK9un5Cs0fRdOq4"
      },
      "elapsed": "0:00:01.077280",
      "text": "{\n  \"args\": {\n    \"foo1\": \"foo1\",\n    \"foo2\": \"foo2\"\n  },\n  \

In [ ]:
!python notebook2script.py Runner.ipynb

Converted Runner.ipynb to exp/nb_Runner.py
